In [25]:
# %pip install langchain_community langchain_google_genai
# %pip install --upgrade --quiet  langchain-google-genai pillow
# %pip install -qU langchain-groq
# %pip install langchain_faiss
# %pip install -qU langchain-cohere
# %pip install faiss-cpu


In [27]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
# from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


# Load environment variables from a .env file
load_dotenv()


from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")
# result = llm.invoke("")
# print(result.content)


In [28]:
import pandas as pd

file_path = ('./dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [29]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()


In [30]:
docs

[Document(metadata={'source': './dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv', 'row': 0}, page_content="flags: B\ninstruction: question about cancelling order {{Order Number}}\ncategory: ORDER\nintent: cancel_order\nresponse: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."),
 Document(metadata={'source': './dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv', 'row': 1}, page_content="flags: BQZ\ninstruction: i have a question about cancelling oorder {{Order Number}}\ncategory: ORDER\nintent: cancel_order\nresponse: I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satis

In [31]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
import faiss

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)


c:\Users\visha\miniconda3\envs\mchatbot\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\visha\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
vector_store.add_documents(documents=docs)

In [33]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-small-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
answer= rag_chain.invoke({"input": "how can i credit in my card?"})
answer['answer']

"I'm sorry, I can't assist with crediting your card. Please contact our customer support team directly for assistance with that issue.  They can be reached at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. \n\n\n\n"